Overview

In [2]:
#importing necessary libraries
import tensorflow as tf
from tensorflow.keras import models, layers, regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_predict
import numpy as np
from scikeras.wrappers import KerasClassifier
import os
import shutil
from sklearn.model_selection import train_test_split

In [5]:
#Importing and splitting the data

class_names = ['glioma_tumor', 'meningioma_tumor', 'pituitary_tumor', 'normal']

train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

def check_initial_counts(class_name):
    images = os.listdir(class_name)
    return len(images)

print("Initial file counts:")
for class_name in class_names:
    count = check_initial_counts(class_name)
    print(f"Class '{class_name}' has {count} images initially.")

split_dirs = ['train', 'val', 'test']
for split_dir in split_dirs:
    os.makedirs(split_dir, exist_ok=True)
    for class_name in class_names:
        os.makedirs(os.path.join(split_dir, class_name), exist_ok=True)

def split_and_move_data(class_name):
    images = os.listdir(class_name)  
    
    train_images, temp_images = train_test_split(images, test_size=(1 - train_ratio), random_state=42)
    
    val_images, test_images = train_test_split(temp_images, test_size=(test_ratio / (val_ratio + test_ratio)), random_state=42)

    for image in train_images:
        shutil.move(os.path.join(class_name, image), os.path.join('train', class_name, image))
    for image in val_images:
        shutil.move(os.path.join(class_name, image), os.path.join('val', class_name, image))
    for image in test_images:
        shutil.move(os.path.join(class_name, image), os.path.join('test', class_name, image))

for class_name in class_names:
    split_and_move_data(class_name)

print("Data has been split into training, validation, and test sets.")

for split_dir in split_dirs:
    print(f"\n{split_dir.upper()} SET:")
    for class_name in class_names:
        dir_path = os.path.join(split_dir, class_name)
        num_images = len(os.listdir(dir_path))
        print(f"Class '{class_name}' has {num_images} images in {split_dir} set.")


Initial file counts:
Class 'glioma_tumor' has 901 images initially.
Class 'meningioma_tumor' has 913 images initially.
Class 'pituitary_tumor' has 844 images initially.
Class 'normal' has 438 images initially.
Data has been split into training, validation, and test sets.

TRAIN SET:
Class 'glioma_tumor' has 720 images in train set.
Class 'meningioma_tumor' has 730 images in train set.
Class 'pituitary_tumor' has 675 images in train set.
Class 'normal' has 350 images in train set.

VAL SET:
Class 'glioma_tumor' has 90 images in val set.
Class 'meningioma_tumor' has 91 images in val set.
Class 'pituitary_tumor' has 84 images in val set.
Class 'normal' has 44 images in val set.

TEST SET:
Class 'glioma_tumor' has 91 images in test set.
Class 'meningioma_tumor' has 92 images in test set.
Class 'pituitary_tumor' has 85 images in test set.
Class 'normal' has 44 images in test set.


In [6]:
# Setting up data generators for training (with augmentation), validation, and test sets, with images at 256x256 resolution

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'train',
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary'
)

val_generator = val_test_datagen.flow_from_directory(
    'val',
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary'
)

test_generator = val_test_datagen.flow_from_directory(
    'test',
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary'
)


Found 2475 images belonging to 4 classes.
Found 309 images belonging to 4 classes.
Found 312 images belonging to 4 classes.
